## Pulp Scheduling


In [83]:
from dataclasses import dataclass
from typing import Tuple, List
import pulp

### Classes


In [84]:
@dataclass(frozen=True)
class Topic:
    name: str
    hours: float


@dataclass(frozen=True)
class Course:
    name: str
    topics: List[Topic]
    final_day: int
    relative_proportion: float

    def total_hours(self) -> float:
        """Sum of all the hours from each topic."""
        total = 0
        for topic in self.topics:
            total += topic.hours
        return total

### Functions


In [85]:
def validate_courses(courses: Tuple[Course], hours_per_day: List[float]) -> None:
    total_proportion = 0
    n_days = len(hours_per_day)

    for course in courses:
        total_proportion += course.relative_proportion

        if course.final_day > n_days:
            raise ValueError(
                f"The final day of {course.name} ({course.final_day}) "
                f"is outside the number of days ({n_days})."
            )

    if total_proportion != 1:
        raise ValueError(f"Relative proportion did not sum to 1 ({total_proportion})")


def perform_course_scale(course:Course, factor: float) -> List[Course]:
    topics = course.topics

    scaled_topics = []
    for topic in topics:
        scaled_hours = topic.hours*factor
        rounded_hours = round(2 * scaled_hours) / 2
        scaled_topic = Topic(topic.name, rounded_hours)
        scaled_topics.append(scaled_topic)
    
    scaled_course = Course(course.name, scaled_topics, course.final_day, course.relative_proportion)
    return scaled_course
    
        

### Testing


In [97]:
# Inputs
# 1)

# Maths
linear_algebra = Topic("Linear Algebra", 1)
geometry = Topic("Geometry", 2)
differential_equations = Topic("Differential Equations", 4)
maths = Course("Maths", [linear_algebra, geometry, differential_equations], 3, 0.3)

# Physics
o_and_w = Topic("Oscillations and Waves", 3)
mechanics = Topic("Mechanics", 3)
quantum_physics = Topic("Quantum Physics", 4)

physics = Course("Physics", [o_and_w, mechanics, quantum_physics], 5, 0.7)

# Creating the schedule
courses = (maths, physics)
hours_per_day = [2, 3, 3, 2, 4, 3]  # Total = 17


# Testing functionality
def scale_courses(courses: List[Course], hours_per_day: List[int]) -> List[Course]:
    validate_courses(courses, hours_per_day)
    # Determine the factor needed to scale each course by to fit the total study hours
    # allocated
    total_study_hours = sum(hours_per_day)
    rescaling_factors = []
    for course in courses:
        scaled_hours = course.relative_proportion * total_study_hours
        rescaling_factor = scaled_hours / course.total_hours()
        rescaling_factors.append(rescaling_factor)

    # Performing the rescale
    rescaled_courses = []
    for course, rescaling_factor in zip(courses, rescaling_factors):
        rescaled_course = perform_course_scale(course, rescaling_factor)
        rescaled_courses.append(rescaled_course)
        
    return rescaled_courses


scaled_courses = scale_courses(courses, hours_per_day)

print(scaled_courses[0].total_hours(), scaled_courses[1].total_hours())

5.0 12.0
